In [305]:
import pandas as pd
import seaborn as sns
import requests
from bs4 import BeautifulSoup
import category_encoders as ce
from geopy.distance import geodesic
from geopy import distance

In [306]:
pd.set_option('display.max_columns', None)

In [307]:
df = pd.read_csv('hotels.csv')
df.head()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 386803 entries, 0 to 386802
Data columns (total 17 columns):
 #   Column                                      Non-Null Count   Dtype  
---  ------                                      --------------   -----  
 0   hotel_address                               386803 non-null  object 
 1   additional_number_of_scoring                386803 non-null  int64  
 2   review_date                                 386803 non-null  object 
 3   average_score                               386803 non-null  float64
 4   hotel_name                                  386803 non-null  object 
 5   reviewer_nationality                        386803 non-null  object 
 6   negative_review                             386803 non-null  object 
 7   review_total_negative_word_counts           386803 non-null  int64  
 8   total_number_of_reviews                     386803 non-null  int64  
 9   positive_review                             386803 non-null  object 
 

In [308]:
drop_list = df[(df['lat'].isnull() == True) | (df['lng'].isnull() == True) ].index.tolist()
len(drop_list)

df = df.drop(drop_list, axis = 0)

In [309]:
columns = df.columns.to_list()
num_cols = list()
cat_cols = list()

for column in columns:
    if df[column].dtype == 'O':
        cat_cols.append(column)
    else:
        num_cols.append(column)

len(cat_cols)
len(num_cols)

9

In [310]:
feature_df = pd.DataFrame()

# Преобразования для ответа на задания на учебной площадке:

# df['hotel_name'].nunique()
# feature_df['review_date'] = pd.to_datetime(df['review_date'])
# feature_df['review_date'].max()

# def rev_func(num_tags):
#     num_tags = num_tags[2:-2]
#     res = num_tags.strip().split(' \', \' ')
#     return res

# df['tags_n'] = df['tags'].apply(rev_func)

# df = df.explode('tags_n')

In [311]:
def get_country(address):
    country = address.split(' ')[-2:]
    country = str(country).replace(',', '').replace("['", "").replace("']", "").replace("'", "")
    if country == 'United Kingdom':
        return 'United Kingdom'
    if country == 'Paris France':
        return 'France'
    if country == 'Amsterdam Netherlands':
        return 'Netherlands'
    if country == 'Milan Italy':
        return 'Italy'
    if country == 'Vienna Austria':
        return 'Austria'
    if country == 'Barcelona Spain':
        return 'Spain'
    else:
        return 'other'
    
df['country'] = df['hotel_address'].apply(get_country)

# url = 'https://www.belastingdienst.nl/wps/wcm/connect/bldcontenten/belastingdienst/individuals/tax-regulations/tax_treaties/list-of-eu-countries/'
# response = requests.get(url)
# page = BeautifulSoup(response.text, 'html.parser')
# eu_countries = page.find('article').find_all('li')
# eu_countries = list(eu_countries)

# for i in range(len(eu_countries)):
#     eu_countries[i] = (str(eu_countries[i])).replace('<li>','').replace('</li>', '').replace('\xa0', ' ').replace('*', '').replace("'", "")
# eu_countries


In [312]:
df['review_date'] = pd.to_datetime(df['review_date'])
df['year'] = df['review_date'].dt.year
df['month'] = df['review_date'].dt.month

def get_season(data):
    if data in [12, 1, 2]:
        return 'winter'
    if data in [3, 4, 5]:
        return 'spring'
    if data in [6, 7, 8]:
        return 'summer'
    if data in [9, 10, 11]:
        return 'autumn'
    
df['season'] = df['month'].apply(get_season)
season_encoder = ce.OneHotEncoder(cols=['season'], use_cat_names=True) # указываем столбец для кодирования
season_bin = season_encoder.fit_transform(df['season'])
feature_df = pd.concat([feature_df, season_bin], axis=1)


In [313]:
df['reviewer_nationality'].nunique()

bin_encoder = ce.BinaryEncoder(cols=['reviewer_nationality']) 
type_bin = bin_encoder.fit_transform(df['reviewer_nationality'])
feature_df = pd.concat([feature_df, type_bin], axis=1)

In [314]:
feature_df['days_since_review'] = df['days_since_review'].apply(lambda x: str(x.split(' ')[:1]).replace("'", '').replace("'", '').replace('[', '').replace(']', ''))
feature_df['days_since_review'] = feature_df['days_since_review'].astype('float64')
                                                         

In [315]:
feature_df['additional_number_of_scoring'] = df['additional_number_of_scoring']
feature_df['average_score'] = round(df['average_score'])
feature_df['review_total_negative_word_counts'] = df['review_total_negative_word_counts']
feature_df['total_number_of_reviews'] = df['total_number_of_reviews']
feature_df['review_total_positive_word_counts'] = df['review_total_positive_word_counts']
feature_df['total_number_of_reviews_reviewer_has_given'] = df['total_number_of_reviews_reviewer_has_given']
feature_df['reviewer_score'] = round(df['reviewer_score'])

In [316]:
import category_encoders as ce
encoder = ce.OneHotEncoder(cols=['country'], use_cat_names=True) # указываем столбец для кодирования
type_bin = encoder.fit_transform(df['country'])
feature_df = pd.concat([feature_df, type_bin], axis=1)


In [317]:
# def if_capital(country, address):
#     if country == 'United Kingdom':
#         if 'London' in address:
#             return 'yes'
#         else:
#             return 'no'
#     if country == 'France':
#         if 'Paris' in address:
#             return 'yes'
#         else:
#             return 'no'
#     if country == 'Netherlands':
#         if 'Amsterdam' in address:
#             return 'yes'
#         else:
#             return 'no'
#     if country == 'Italy':
#         if 'Milan' in address:
#             return 'yes'
#         else:
#             return 'no'
#     if country == 'Austria':
#         if 'Vienna' in address:
#             return 'yes'
#         else:
#             return 'no'
#     if country == 'Spain':
#         if 'Barcelona' in address:
#             return 'yes'
#         else:
#             return 'no'

# df['if_capital'] = df.apply(lambda row: if_capital(row['country'], row['hotel_address']), axis = 1)

# encodery = ce.OneHotEncoder(cols=['if_capital'], use_cat_names=True) 
# type_biny = encodery.fit_transform(df['if_capital'])
# feature_df = pd.concat([feature_df, type_biny], axis=1)

In [318]:
# df['tags'] = df['tags'].apply(lambda x: x.replace("' ", "'").replace(" '", "'").replace("[", "").replace("]", "").split(','))

In [319]:
def get_coordinate(lat, lng):
    return (float(lat), float(lng))

def capital_coordinate(country):
    if country == 'United Kingdom':
        return (float(51.50936092785646), float(-0.12416095345639586))
    if country == 'France':
        return (float(48.853229), float(2.342797))
    if country == 'Netherlands':
        return (float(52.37176471692262), float(4.895930820099642))
    if country == 'Italy':
        return (float(45.46986), float(9.18499))
    if country == 'Austria':
        return (float(48.20471741160276), float(16.37049992180316))
    if country == 'Spain':
        return (float(41.395704279217775), float(2.172020436193314))
        


df['hotel_coordinate'] = df.apply(lambda row: get_coordinate(row['lat'], row['lng']), axis = 1)
df['center_coordinate'] = df['country'].apply(capital_coordinate)


In [320]:
from geopy import distance

def dist_from_cent(hotel_coordinate, center_coordinate):
    return geodesic(hotel_coordinate, center_coordinate).kilometers

df['dist_from_cent_km'] = df.apply(lambda row: dist_from_cent(row['hotel_coordinate'], row['center_coordinate']),  axis = 1)

In [321]:
def city(country):
    if country == 'United Kingdom':
        return 'London'
    if country == 'France':
        return 'Paris'
    if country == 'Netherlands':
        return 'Amsterdam'
    if country == 'Italy':
        return 'Milan'
    if country == 'Austria':
        return 'Vienna'
    if country == 'Spain':
        return 'Barcelona'

df['city'] = df['country'].apply(city)

In [322]:
season_encoder = ce.OneHotEncoder(cols=['season'], use_cat_names=True) # указываем столбец для кодирования
season_bin = season_encoder.fit_transform(df['season'])
feature_df = pd.concat([feature_df, season_bin], axis=1)

In [325]:
feature_df

,season_winter,season_autumn,season_spring,season_summer,reviewer_nationality_0,reviewer_nationality_1,reviewer_nationality_2,reviewer_nationality_3,reviewer_nationality_4,reviewer_nationality_5,reviewer_nationality_6,reviewer_nationality_7,days_since_review,additional_number_of_scoring,average_score,review_total_negative_word_counts,total_number_of_reviews,review_total_positive_word_counts,total_number_of_reviews_reviewer_has_given,reviewer_score,country_United Kingdom,country_France,country_Netherlands,country_Italy,country_Austria,country_Spain,season_winter,season_autumn,season_spring,season_summer
0,1,0,0,0,0,0,0,0,0,0,0,1,531.0,581,8.0,3,1994,4,7,10.0,1,0,0,0,0,0,1,0,0,0
1,1,0,0,0,0,0,0,0,0,0,0,1,203.0,299,8.0,3,1361,2,14,6.0,1,0,0,0,0,0,1,0,0,0
2,0,1,0,0,0,0,0,0,0,0,1,0,289.0,32,9.0,6,406,0,14,8.0,0,1,0,0,0,0,0,1,0,0
3,0,1,0,0,0,0,0,0,0,0,0,1,681.0,34,8.0,0,607,11,8,10.0,0,1,0,0,0,0,0,1,0,0
4,0,0,1,0,0,0,0,0,0,0,1,1,516.0,914,8.0,4,7586,20,10,10.0,0,0,1,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
386798,0,0,1,0,0,0,0,0,0,1,0,0,106.0,107,9.0,0,617,10,10,9.0,1,0,0,0,0,0,0,0,1,0
386799,1,0,0,0,0,0,1,0,1,0,1,1,171.0,272,8.0,0,3224,93,1,9.0,0,0,0,0,1,0,1,0,0,0
386800,1,0,0,0,0,0,0,0,1,1,1,1,543.0,457,7.0,12,2751,9,21,8.0,1,0,0,0,0,0,1,0,0,0
386801,0,0,1,0,0,0,0,0,1,0,0,0,74.0,365,8.0,0,1567,6,28,9.0,1,0,0,0,0,0,0,0,1,0


In [323]:
feature_df.corr()

,season_winter,season_autumn,season_spring,season_summer,reviewer_nationality_0,reviewer_nationality_1,reviewer_nationality_2,reviewer_nationality_3,reviewer_nationality_4,reviewer_nationality_5,reviewer_nationality_6,reviewer_nationality_7,days_since_review,additional_number_of_scoring,average_score,review_total_negative_word_counts,total_number_of_reviews,review_total_positive_word_counts,total_number_of_reviews_reviewer_has_given,reviewer_score,country_United Kingdom,country_France,country_Netherlands,country_Italy,country_Austria,country_Spain,season_winter,season_autumn,season_spring,season_summer
season_winter,1.000000,-0.306919,-0.320443,-0.340954,-0.004202,-0.002197,-0.027475,-0.001313,-0.044729,-0.060545,0.003667,0.057374,0.044055,0.029792,0.006929,-0.012548,0.009997,-0.012013,0.007885,0.028512,0.057719,-0.026671,0.003860,-0.035353,0.007220,-0.044217,1.000000,-0.306919,-0.320443,-0.340954
season_autumn,-0.306919,1.000000,-0.324512,-0.345283,0.005160,-0.000619,0.024695,0.012622,0.019757,0.032307,0.005268,-0.027788,0.294462,-0.009059,-0.008765,-0.011544,-0.003966,-0.033921,0.014404,-0.033578,-0.015364,0.015634,-0.012001,0.007817,0.002106,0.012107,-0.306919,1.000000,-0.324512,-0.345283
season_spring,-0.320443,-0.324512,1.000000,-0.360497,-0.002928,-0.003077,-0.026350,-0.000890,-0.013806,-0.027694,-0.005789,0.029233,-0.176563,0.007916,0.000944,0.009948,0.000416,0.020340,-0.006087,0.011681,0.010090,-0.006719,0.007205,-0.000422,-0.018332,-0.001103,-0.320443,-0.324512,1.000000,-0.360497
season_summer,-0.340954,-0.345283,-0.360497,1.000000,0.001907,0.005651,0.028060,-0.009889,0.036853,0.053345,-0.002844,-0.056141,-0.149871,-0.027200,0.000872,0.013149,-0.006071,0.023812,-0.015216,-0.006352,-0.049678,0.016844,0.000762,0.026349,0.008991,0.031301,-0.340954,-0.345283,-0.360497,1.000000
reviewer_nationality_0,-0.004202,0.005160,-0.002928,0.001907,1.000000,0.011581,0.008432,0.034000,0.031001,0.016881,0.039031,-0.019853,0.000571,-0.010312,-0.004163,-0.007029,-0.004639,-0.006939,0.000185,-0.006064,-0.018223,0.016942,0.000788,0.005238,0.003130,0.003974,-0.004202,0.005160,-0.002928,0.001907
reviewer_nationality_1,-0.002197,-0.000619,-0.003077,0.005651,0.011581,1.000000,-0.036355,0.045460,0.061252,0.054643,0.291994,-0.075977,-0.013580,-0.050323,-0.013045,-0.007999,-0.003318,-0.012320,0.037674,-0.025088,-0.096132,0.028335,0.007360,0.057166,0.071401,0.010980,-0.002197,-0.000619,-0.003077,0.005651
reviewer_nationality_2,-0.027475,0.024695,-0.026350,0.028060,0.008432,-0.036355,1.000000,0.217459,0.101355,0.288142,0.145745,-0.331478,0.002661,-0.102681,-0.000707,-0.017671,-0.022413,-0.016727,0.106064,-0.032250,-0.174293,0.053335,0.011645,0.089408,0.108742,0.047629,-0.027475,0.024695,-0.026350,0.028060
reviewer_nationality_3,-0.001313,0.012622,-0.000890,-0.009889,0.034000,0.045460,0.217459,1.000000,0.144903,0.216518,0.281393,-0.184195,0.003800,-0.082520,-0.019499,-0.000108,-0.000730,-0.014668,0.103404,-0.051131,-0.156787,0.033164,0.032400,0.077653,0.111321,0.027564,-0.001313,0.012622,-0.000890,-0.009889
reviewer_nationality_4,-0.044729,0.019757,-0.013806,0.036853,0.031001,0.061252,0.101355,0.144903,1.000000,0.276098,0.293698,-0.435733,-0.008355,-0.138645,0.037083,-0.000260,-0.051443,0.030251,0.071929,-0.012485,-0.225417,0.083869,0.046462,0.082726,0.119936,0.059045,-0.044729,0.019757,-0.013806,0.036853
reviewer_nationality_5,-0.060545,0.032307,-0.027694,0.053345,0.016881,0.054643,0.288142,0.216518,0.276098,1.000000,0.141761,-0.451580,-0.008671,-0.134125,0.030996,-0.010295,-0.054660,0.019903,0.081591,-0.011977,-0.220409,0.113756,0.028556,0.091937,0.071141,0.070547,-0.060545,0.032307,-0.027694,0.053345


In [324]:
X = feature_df.drop(['reviewer_score'], axis = 1)  
y = feature_df['reviewer_score'] 

from sklearn.model_selection import train_test_split  

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Импортируем необходимые библиотеки:  
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели  
from sklearn import metrics # инструменты для оценки точности модели  
  
# Создаём модель  
regr = RandomForestRegressor(n_estimators=100)  
      
# Обучаем модель на тестовом наборе данных  
regr.fit(X_train, y_train)  
      
# Используем обученную модель для предсказания рейтинга отелей в тестовой выборке.  
# Предсказанные значения записываем в переменную y_pred  
y_pred = regr.predict(X_test)  

print('MAPE:', metrics.mean_absolute_percentage_error(y_test, y_pred))



MAPE: 0.14620131644172366
